<a href="https://colab.research.google.com/github/anoopjakob/DLA-Assignment/blob/main/object_detection_loss_curves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Uploading Road sign dataset

In [14]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anoopjacob","key":"2bcedcd5c87d722458b4db197e216543"}'}

In [15]:
!pip install -q kaggle

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [29]:
cd /content/

/content


In [34]:
!kaggle datasets download -d andrewmvd/road-sign-detection

 97% 212M/218M [00:01<00:00, 147MB/s]
100% 218M/218M [00:01<00:00, 144MB/s]


### Cloning Yolov5

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo

Cloning into 'yolov5'...
remote: Enumerating objects: 8802, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 8802 (delta 22), reused 17 (delta 10), pack-reused 8754
Receiving objects: 100% (8802/8802), 9.38 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (6109/6109), done.


In [10]:
cd /content/yolov5

/content/yolov5


In [11]:
! pip install -r requirements.txt  # install

     |████████████████████████████████| 636 kB 7.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Installing dependencies

In [21]:
!pip install -qr requirements.txt
!pip install split-folders

In [22]:
import torch
import splitfolders

# To display images
from IPython.display import Image, clear_output 

print('Setup complete.')
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU')) 


Setup complete.
Using torch 1.9.0+cu102 CPU


In [39]:
## unzipping road sign zip file
%cd /content
!unzip road-sign-detection.zip

/content
Archive:  road-sign-detection.zip
replace annotations/road0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [49]:
# splitting it to train and validation

splitfolders.ratio("/content/export", output="", seed=1337, ratio=(.35, .15, .5), group_prefix=None) 


Copying files: 0 files [00:00, ? files/s]
Copying files: 98 files [00:00, 977.51 files/s]
Copying files: 234 files [00:00, 1067.24 files/s]
Copying files: 360 files [00:00, 1118.27 files/s]
Copying files: 481 files [00:00, 1141.21 files/s]
Copying files: 574 files [00:00, 1058.94 files/s]
Copying files: 666 files [00:00, 986.94 files/s] 
Copying files: 756 files [00:00, 832.92 files/s]
Copying files: 838 files [00:00, 825.31 files/s]
Copying files: 920 files [00:01, 725.15 files/s]
Copying files: 1754 files [00:01, 1508.07 files/s]


### Model Configuration

In [78]:
%cat /content/data.yaml

train: /content/train
val: /content/val

nc: 4
names: ['Trafic Light','Stop', 'Speedlimit','Crosswalk']


In [79]:
# define number of classes based on YAML
import yaml
with open("/content/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [80]:
type(num_classes)

str

In [81]:
%cat /content/yolov5/models/yolov5s.yaml

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, Fa

In [82]:
#customize iPython writefile to write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [83]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### Training

In [84]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data '/content/data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=/content/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-358-g3e7c59a torch 1.9.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sca

In [70]:
! rm -rf /content/export/annotations

In [71]:
! rm -rf /content/export/images

In [76]:
! mv /content/export/val /content